In [2]:
!pip install shap

  Using cached slicer-0.0.8-py3-none-any.whl.metadata (4.0 kB)
   ---------------------------------------- 0.0/545.3 kB ? eta -:--:--
   --------------------------------------- 545.3/545.3 kB 17.7 MB/s eta 0:00:00
Using cached slicer-0.0.8-py3-none-any.whl (15 kB)


In [4]:
!pip install lime

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283840 sha256=4c548a1ac6aa3ca1306179f7f627374980f0a76fd051f63b6eedb5ff615e89dd
  Stored in directory: c:\users\02\appdata\local\pip\cache\wheels\e7\5d\0e\4b4fff9a47468fed5633211fb3b76d1db43fe806a17fb7486a
Successfully built lime


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime
import os
import shap
from lime import lime_tabular

# 생존 분석 라이브러리
from lifelines import CoxPHFitter, KaplanMeierFitter
from lifelines.statistics import logrank_test
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.ensemble import RandomSurvivalForest
from sksurv.ensemble import GradientBoostingSurvivalAnalysis
from sksurv.metrics import concordance_index_censored

# 머신러닝 라이브러리
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
import xgboost as xgb
import joblib

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')

def setup_korean_font():
    """한글 폰트 설정"""
    import platform
    import matplotlib.font_manager as fm
    
    system = platform.system()
    
    if system == 'Windows':
        try:
            plt.rcParams['font.family'] = 'Malgun Gothic'
        except:
            try:
                font_path = 'C:/Windows/Fonts/malgun.ttf'
                font_name = fm.FontProperties(fname=font_path).get_name()
                plt.rc('font', family=font_name)
            except:
                print("⚠️ 한글 폰트 설정 실패")
    elif system == 'Darwin':
        plt.rcParams['font.family'] = 'AppleGothic'
    else:
        plt.rcParams['font.family'] = 'NanumGothic'
    
    plt.rcParams['axes.unicode_minus'] = False
    print("✅ 한글 폰트 설정 완료")

class LiverCancerRiskStratification:
    """간암 위험도 계층화 모델 클래스 (XAI 포함)"""
    
    def __init__(self, data_path):
        setup_korean_font()
        self.data_path = data_path
        self.df = None
        self.processed_df = None
        self.models = {}
        self.results = {}
        self.feature_names = []
        self.risk_scores = {}
        self.shap_explainers = {}
        self.shap_values = {}
        self.lime_explainers = {}
        
        print(f"🚀 간암 위험도 계층화 모델 초기화 (XAI 포함)")
        print(f"📁 데이터 경로: {data_path}")
        print(f"⏰ 시작 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print("="*60)
    
    def load_and_explore_data(self):
        """데이터 로드 및 탐색적 분석"""
        print("\n📊 1. 데이터 로드 및 탐색")
        
        try:
            self.df = pd.read_csv(self.data_path)
            print(f"✅ 데이터 로드 성공: {self.df.shape[0]}행 × {self.df.shape[1]}열")
        except Exception as e:
            print(f"❌ 데이터 로드 실패: {e}")
            return False
        
        # 기본 정보 출력
        print(f"📈 데이터 기본 정보:")
        print(f"   - 총 환자 수: {len(self.df)}")
        print(f"   - 총 컬럼 수: {len(self.df.columns)}")
        
        # 생존 상태 분포
        if 'vital_status' in self.df.columns:
            status_counts = self.df['vital_status'].value_counts()
            print(f"   - 생존 환자: {status_counts.get('Alive', 0)}명")
            print(f"   - 사망 환자: {status_counts.get('Dead', 0)}명")
            print(f"   - 사망률: {status_counts.get('Dead', 0)/len(self.df)*100:.1f}%")
        
        return True
    
    def preprocess_data(self):
        """데이터 전처리"""
        print("\n🔧 2. 데이터 전처리")
        
        # 위험도 계층화 모델용 선택된 컬럼들
        selected_columns = [
            # 생존 결과 변수
            'vital_status', 'days_to_death', 'days_to_last_follow_up',
            # 간기능 평가 (가장 중요)
            'child_pugh_classification', 'ishak_fibrosis_score',
            # 종양 특성 및 병기
            'ajcc_pathologic_stage', 'ajcc_pathologic_t', 'ajcc_pathologic_n', 'ajcc_pathologic_m',
            'tumor_grade', 'morphology',
            # 환자 기본 특성
            'age_at_diagnosis', 'gender', 'race', 'ethnicity',
            # 치료 이력
            'prior_treatment', 'prior_malignancy', 'synchronous_malignancy',
            # 핵심 변수들
            'residual_disease', 'classification_of_tumor', 'primary_diagnosis', 'year_of_diagnosis'
        ]
        
        # 존재하는 컬럼만 선택
        available_columns = [col for col in selected_columns if col in self.df.columns]
        missing_columns = [col for col in selected_columns if col not in self.df.columns]
        
        print(f"✅ 사용 가능한 컬럼: {len(available_columns)}개")
        if missing_columns:
            print(f"⚠️  누락된 컬럼: {missing_columns}")
        
        self.processed_df = self.df[available_columns].copy()
        
        # 생존 시간 및 이벤트 변수 생성
        print("🔄 생존 변수 생성 중...")
        self.processed_df['event'] = (self.processed_df['vital_status'] == 'Dead').astype(int)
        
        # 생존 시간 계산
        self.processed_df['duration'] = self.processed_df['days_to_death'].fillna(
            self.processed_df['days_to_last_follow_up']
        )
        
        # 유효하지 않은 생존 시간 제거
        valid_mask = (self.processed_df['duration'].notna()) & (self.processed_df['duration'] > 0)
        self.processed_df = self.processed_df[valid_mask].copy()
        
        print(f"✅ 유효한 생존 데이터: {len(self.processed_df)}명")
        print(f"   - 사망 이벤트: {self.processed_df['event'].sum()}건")
        print(f"   - 중간 생존 시간: {self.processed_df['duration'].median():.0f}일")
        
        # 결측값 분석
        print("\n📋 결측값 분석:")
        missing_analysis = self.processed_df.isnull().sum()
        missing_percent = (missing_analysis / len(self.processed_df) * 100).round(1)
        
        for col in missing_analysis[missing_analysis > 0].index:
            print(f"   - {col}: {missing_analysis[col]}개 ({missing_percent[col]}%)")
        
        # 높은 결측률 컬럼 제거 (80% 이상)
        high_missing_cols = missing_percent[missing_percent > 80].index.tolist()
        if high_missing_cols:
            print(f"🗑️  높은 결측률 컬럼 제거: {high_missing_cols}")
            self.processed_df = self.processed_df.drop(columns=high_missing_cols)
        
        return True
    
    def prepare_features(self):
        """특성 준비 및 인코딩 (NA와 Unknown 값 처리 포함)"""
        print("\n🎯 3. 특성 준비 및 인코딩")
        
        # 특성과 타겟 분리
        feature_cols = [col for col in self.processed_df.columns 
                       if col not in ['vital_status', 'days_to_death', 'days_to_last_follow_up', 
                                     'event', 'duration']]
        
        X = self.processed_df[feature_cols].copy()
        y_duration = self.processed_df['duration'].values
        y_event = self.processed_df['event'].values.astype(bool)
        
        print(f"📊 초기 특성 개수: {len(feature_cols)}")
        print(f"📊 샘플 개수: {len(X)}")
        
        # 범주형 변수 인코딩
        categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
        numerical_cols = X.select_dtypes(include=[np.number]).columns.tolist()
        
        print(f"🔤 범주형 변수: {len(categorical_cols)}개")
        print(f"🔢 수치형 변수: {len(numerical_cols)}개")
        
        # 결측값 처리
        print("🔄 결측값 처리 중...")
        
        # 수치형 변수: 중앙값으로 대체
        if numerical_cols:
            num_imputer = SimpleImputer(strategy='median')
            X[numerical_cols] = num_imputer.fit_transform(X[numerical_cols])
        
        # 임상적으로 의미있는 Unknown 값을 가질 수 있는 컬럼들
        meaningful_unknown_cols = [
            'child_pugh_classification', 'ajcc_pathologic_stage', 'ajcc_pathologic_t',
            'ajcc_pathologic_n', 'ajcc_pathologic_m', 'tumor_grade', 'residual_disease',
            'prior_malignancy', 'synchronous_malignancy', 'prior_treatment'
        ]
        
        label_encoders = {}
        
        for col in categorical_cols:
            if col in X.columns:
                print(f"\n   🔍 {col} 처리:")
                
                # 현재 값 분포 확인
                value_counts = X[col].value_counts(dropna=False)
                print(f"      - 전처리 전 분포: {dict(list(value_counts.items())[:3])}")
                
                # 'NA' 문자열을 결측치로 변환
                if 'NA' in X[col].values:
                    X[col] = X[col].replace('NA', np.nan)
                    print(f"      - 'NA' 문자열을 결측치로 변환")
                
                # Unknown 값 처리 결정
                has_unknown = X[col].str.contains('Unknown', na=False).any() if X[col].dtype == object else False
                
                if has_unknown:
                    if col in meaningful_unknown_cols:
                        print(f"      - 'Unknown' 값 유지 (임상적 의미 있음)")
                        if X[col].isnull().any():
                            mode_value = X[col].mode()
                            if not mode_value.empty:
                                fill_value = mode_value[0]
                                X[col] = X[col].fillna(fill_value)
                                print(f"      - 결측치를 '{fill_value}'로 대체")
                    else:
                        print(f"      - 'Unknown' 값을 결측치로 변환 후 대체")
                        X[col] = X[col].replace('Unknown', np.nan)
                        if X[col].isnull().any():
                            mode_value = X[col].mode()
                            if not mode_value.empty:
                                fill_value = mode_value[0]
                                X[col] = X[col].fillna(fill_value)
                                print(f"      - 결측치를 '{fill_value}'로 대체")
                else:
                    if X[col].isnull().any():
                        mode_value = X[col].mode()
                        if not mode_value.empty:
                            fill_value = mode_value[0]
                            X[col] = X[col].fillna(fill_value)
                            print(f"      - 결측치를 '{fill_value}'로 대체")
        
        # 모든 범주형 변수 인코딩
        print("\n🔄 범주형 변수 인코딩:")
        all_categorical_cols = X.select_dtypes(include=['object']).columns.tolist()
        for col in all_categorical_cols:
            le = LabelEncoder()
            X[col] = le.fit_transform(X[col].astype(str))
            label_encoders[col] = le
            
            if col in meaningful_unknown_cols:
                mapping = dict(zip(le.classes_, le.transform(le.classes_)))
                print(f"   - {col} 인코딩 매핑: {mapping}")
        
        # 특성 스케일링
        scaler = StandardScaler()
        X_scaled = pd.DataFrame(
            scaler.fit_transform(X),
            columns=X.columns,
            index=X.index
        )
        
        self.feature_names = X_scaled.columns.tolist()
        
        # scikit-survival 형식으로 변환
        y_structured = np.array([(event, duration) for event, duration in zip(y_event, y_duration)],
                               dtype=[('event', '?'), ('time', '<f8')])
        
        print("✅ 특성 준비 완료")
        
        return X_scaled, y_structured, y_duration, y_event, scaler, label_encoders
    
    def create_risk_scores(self, X):
        """위험도 점수 계산"""
        print("\n🎯 4. 위험도 점수 계산")
        
        # 원본 데이터에서 주요 변수들 추출
        risk_df = self.processed_df.copy()
        
        # Child-Pugh 점수 계산
        child_pugh_scores = np.zeros(len(risk_df))
        if 'child_pugh_classification' in risk_df.columns:
            for i, val in enumerate(risk_df['child_pugh_classification']):
                if pd.isna(val) or val == 'Unknown':
                    child_pugh_scores[i] = 1  # 중간값
                elif 'A' in str(val):
                    child_pugh_scores[i] = 0
                elif 'B' in str(val):
                    child_pugh_scores[i] = 2
                elif 'C' in str(val):
                    child_pugh_scores[i] = 4
                else:
                    child_pugh_scores[i] = 1
        
        # AJCC 병기 점수 계산
        stage_scores = np.zeros(len(risk_df))
        if 'ajcc_pathologic_stage' in risk_df.columns:
            for i, val in enumerate(risk_df['ajcc_pathologic_stage']):
                if pd.isna(val) or val == 'Unknown':
                    stage_scores[i] = 1  # 중간값
                elif 'I' in str(val) and 'II' not in str(val) and 'III' not in str(val) and 'IV' not in str(val):
                    stage_scores[i] = 0
                elif 'II' in str(val) and 'III' not in str(val):
                    stage_scores[i] = 1
                elif 'III' in str(val) and 'IV' not in str(val):
                    stage_scores[i] = 2
                elif 'IV' in str(val):
                    stage_scores[i] = 3
                else:
                    stage_scores[i] = 1
        
        # 연령 점수 계산
        age_scores = np.zeros(len(risk_df))
        if 'age_at_diagnosis' in risk_df.columns:
            ages = risk_df['age_at_diagnosis'].fillna(risk_df['age_at_diagnosis'].median())
            age_scores = np.where(ages < 60, 0, np.where(ages <= 70, 1, 2))
        
        # 성별 점수 계산
        gender_scores = np.zeros(len(risk_df))
        if 'gender' in risk_df.columns:
            gender_scores = np.where(risk_df['gender'] == 'male', 1, 0)
        
        # 총 위험도 점수 계산
        total_scores = child_pugh_scores + stage_scores + age_scores + gender_scores
        
        # 3단계 위험度 분류
        risk_categories = np.where(total_scores <= 2, 0,  # 저위험군
                                 np.where(total_scores <= 5, 1, 2))  # 중위험군, 고위험군
        
        risk_labels = ['저위험군', '중위험군', '고위험군']
        
        print(f"📊 위험도 분포:")
        for i, label in enumerate(risk_labels):
            count = np.sum(risk_categories == i)
            percentage = count / len(risk_categories) * 100
            print(f"   - {label}: {count}명 ({percentage:.1f}%)")
        
        return total_scores, risk_categories, risk_labels
    
    def split_data(self, X, y_structured, y_duration, y_event, risk_categories):
        """데이터 분할"""
        print("\n✂️  5. 데이터 분할 (훈련:검증:테스트 = 60:20:20)")
        
        # 먼저 훈련+검증 vs 테스트로 분할
        X_temp, X_test, y_temp_struct, y_test_struct, y_temp_dur, y_test_dur, y_temp_event, y_test_event, risk_temp, risk_test = \
            train_test_split(X, y_structured, y_duration, y_event, risk_categories,
                           test_size=0.2, random_state=42, stratify=risk_categories)
        
        # 훈련 vs 검증으로 분할
        X_train, X_val, y_train_struct, y_val_struct, y_train_dur, y_val_dur, y_train_event, y_val_event, risk_train, risk_val = \
            train_test_split(X_temp, y_temp_struct, y_temp_dur, y_temp_event, risk_temp,
                           test_size=0.25, random_state=42, stratify=risk_temp)
        
        print(f"📊 훈련 세트: {len(X_train)}명 (사망: {y_train_event.sum()}명)")
        print(f"📊 검증 세트: {len(X_val)}명 (사망: {y_val_event.sum()}명)")
        print(f"📊 테스트 세트: {len(X_test)}명 (사망: {y_test_event.sum()}명)")
        
        return (X_train, X_val, X_test, 
                y_train_struct, y_val_struct, y_test_struct,
                y_train_dur, y_val_dur, y_test_dur,
                y_train_event, y_val_event, y_test_event,
                risk_train, risk_val, risk_test)
    
    def train_models(self, X_train, X_val, X_test, 
                    y_train_struct, y_val_struct, y_test_struct,
                    y_train_dur, y_val_dur, y_test_dur,
                    y_train_event, y_val_event, y_test_event,
                    risk_train, risk_val, risk_test):
        """모델 훈련 (안정적인 모델들로 구성)"""
        print("\n🤖 6. 모델 훈련")
        
        # 1. Cox 비례위험 모델 (생존 예측)
        print("🔄 Cox 비례위험 모델 훈련 중...")
        try:
            cox_model = CoxPHSurvivalAnalysis(alpha=0.5)
            cox_model.fit(X_train, y_train_struct)
            self.models['Cox_Survival'] = cox_model
            print("✅ Cox 생존 모델 훈련 완료")
        except Exception as e:
            print(f"❌ Cox 모델 훈련 실패: {e}")
        
        # 2. Random Survival Forest
        print("🔄 Random Survival Forest 훈련 중...")
        try:
            rsf_model = RandomSurvivalForest(
                n_estimators=100,
                max_depth=10,
                min_samples_split=10,
                min_samples_leaf=5,
                random_state=42,
                n_jobs=-1
            )
            rsf_model.fit(X_train, y_train_struct)
            self.models['RSF'] = rsf_model
            print("✅ Random Survival Forest 훈련 완료")
        except Exception as e:
            print(f"❌ RSF 모델 훈련 실패: {e}")
        
        # 3. Gradient Boosting Survival Analysis
        print("🔄 Gradient Boosting Survival Analysis 훈련 중...")
        try:
            gbsa_model = GradientBoostingSurvivalAnalysis(
                n_estimators=100,
                learning_rate=0.1,
                max_depth=3,
                random_state=42
            )
            gbsa_model.fit(X_train, y_train_struct)
            self.models['GBSA'] = gbsa_model
            print("✅ Gradient Boosting Survival Analysis 훈련 완료")
        except Exception as e:
            print(f"❌ GBSA 모델 훈련 실패: {e}")
        
        # 4. 위험도 분류 모델 (Random Forest Classifier)
        print("🔄 위험도 분류 모델 (Random Forest) 훈련 중...")
        try:
            rf_classifier = RandomForestClassifier(
                n_estimators=100,
                max_depth=10,
                random_state=42,
                n_jobs=-1
            )
            rf_classifier.fit(X_train, risk_train)
            self.models['Risk_RF'] = rf_classifier
            print("✅ 위험도 분류 모델 (Random Forest) 훈련 완료")
        except Exception as e:
            print(f"❌ 위험도 분류 모델 (Random Forest) 훈련 실패: {e}")
        
        # 5. 로지스틱 회귀 (위험도 분류)
        print("🔄 로지스틱 회귀 모델 훈련 중...")
        try:
            lr_model = LogisticRegression(
                multi_class='multinomial',
                solver='lbfgs',
                random_state=42,
                max_iter=1000
            )
            lr_model.fit(X_train, risk_train)
            self.models['Logistic_Risk'] = lr_model
            print("✅ 로지스틱 회귀 모델 훈련 완료")
        except Exception as e:
            print(f"❌ 로지스틱 회귀 모델 훈련 실패: {e}")
        
        # 6. XGBoost 분류 모델
        print("🔄 XGBoost 분류 모델 훈련 중...")
        try:
            xgb_model = xgb.XGBClassifier(
                n_estimators=100,
                max_depth=6,
                learning_rate=0.1,
                random_state=42,
                eval_metric='mlogloss'
            )
            xgb_model.fit(X_train, risk_train)
            self.models['XGBoost_Risk'] = xgb_model
            print("✅ XGBoost 분류 모델 훈련 완료")
        except Exception as e:
            print(f"❌ XGBoost 분류 모델 훈련 실패: {e}")
        
        print(f"\n🎯 총 {len(self.models)}개 모델 훈련 완료")
        print("ℹ️  DeepSurv 모델은 안정성을 위해 제외됨")
        
        return True
    
    def evaluate_models(self, X_train, X_val, X_test,
                       y_train_struct, y_val_struct, y_test_struct,
                       y_train_dur, y_val_dur, y_test_dur,
                       y_train_event, y_val_event, y_test_event,
                       risk_train, risk_val, risk_test):
        """모델 평가"""
        print("\n📈 7. 모델 평가")
        
        datasets = {
            'Train': (X_train, y_train_struct, y_train_dur, y_train_event, risk_train),
            'Validation': (X_val, y_val_struct, y_val_dur, y_val_event, risk_val),
            'Test': (X_test, y_test_struct, y_test_dur, y_test_event, risk_test)
        }
        
        for model_name, model in self.models.items():
            print(f"\n🔍 {model_name} 모델 평가:")
            self.results[model_name] = {}
            
            for dataset_name, (X, y_struct, y_dur, y_event, risk_true) in datasets.items():
                try:
                    if 'Survival' in model_name or model_name in ['RSF', 'GBSA']:
                        # 생존 모델 평가
                        risk_scores = model.predict(X)
                        c_index = concordance_index_censored(y_struct['event'], y_struct['time'], risk_scores)[0]
                        self.results[model_name][dataset_name] = {'c_index': c_index}
                        print(f"   {dataset_name}: C-index = {c_index:.3f}")
                    
                    else:
                        # 분류 모델 평가
                        risk_pred = model.predict(X)
                        accuracy = (risk_pred == risk_true).mean()
                        
                        # 다중클래스 AUC 계산
                        try:
                            risk_proba = model.predict_proba(X)
                            auc_score = roc_auc_score(risk_true, risk_proba, multi_class='ovr', average='weighted')
                        except:
                            auc_score = np.nan
                        
                        self.results[model_name][dataset_name] = {
                            'accuracy': accuracy,
                            'auc': auc_score
                        }
                        print(f"   {dataset_name}: Accuracy = {accuracy:.3f}, AUC = {auc_score:.3f}")
                    
                except Exception as e:
                    print(f"   ❌ {dataset_name} 평가 실패: {e}")
                    self.results[model_name][dataset_name] = {'error': str(e)}
        
        return True
    
    def explain_models(self, X_train, X_test):
        """XAI 모델 설명 생성 (수정된 버전)"""
        print("\n🔍 XAI 모델 설명 생성")
        
        # SHAP 설명기 초기화
        print("🔄 SHAP 설명 생성 중...")
        for model_name in ['Risk_RF', 'XGBoost_Risk']:
            if model_name in self.models:
                try:
                    # 더 작은 샘플로 SHAP 계산 (메모리 효율성)
                    X_test_sample = X_test.iloc[:50]  # 처음 50개 샘플만 사용
                    
                    explainer = shap.TreeExplainer(self.models[model_name])
                    shap_values = explainer.shap_values(X_test_sample)
                    
                    print(f"   - {model_name} SHAP 값 형태: {np.array(shap_values).shape if isinstance(shap_values, list) else shap_values.shape}")
                    
                    self.shap_explainers[model_name] = explainer
                    self.shap_values[model_name] = shap_values
                    print(f"✅ {model_name} SHAP 설명 생성 완료")
                except Exception as e:
                    print(f"❌ {model_name} SHAP 실패: {e}")
        
        # LIME 설명기 초기화
        print("\n🔄 LIME 설명 생성 중...")
        class_names = ['저위험군', '중위험군', '고위험군']
        for model_name in ['Risk_RF', 'XGBoost_Risk']:
            if model_name in self.models:
                try:
                    explainer = lime_tabular.LimeTabularExplainer(
                        training_data=X_train.values,
                        feature_names=self.feature_names,
                        class_names=class_names,
                        mode='classification',
                        discretize_continuous=True
                    )
                    self.lime_explainers[model_name] = explainer
                    print(f"✅ {model_name} LIME 설명기 생성 완료")
                except Exception as e:
                    print(f"❌ {model_name} LIME 실패: {e}")
        
        return True
    
    def generate_xai_visualizations(self, X_test, sample_index=0):
        """XAI 시각화 생성 및 저장 (수정된 버전)"""
        print("\n📊 XAI 시각화 생성")
        
        # SHAP 시각화
        shap_figures = []
        for model_name in self.shap_explainers:
            try:
                print(f"🔄 {model_name} SHAP 시각화 디버깅:")
                
                # SHAP 값 구조 확인
                shap_vals = self.shap_values[model_name]
                print(f"   - SHAP 값 타입: {type(shap_vals)}")
                print(f"   - SHAP 값 형태: {np.array(shap_vals).shape if isinstance(shap_vals, list) else shap_vals.shape}")
                print(f"   - 특성명 개수: {len(self.feature_names)}")
                
                # 다중클래스 분류의 경우 클래스별 SHAP 값 처리
                if isinstance(shap_vals, list) and len(shap_vals) == 3:  # 3개 클래스
                    # 모든 클래스의 절댓값 평균으로 요약
                    combined_shap = np.mean([np.abs(shap_vals[i]) for i in range(3)], axis=0)
                    print(f"   - 결합된 SHAP 형태: {combined_shap.shape}")
                else:
                    combined_shap = shap_vals
                
                # Summary plot (막대 그래프)
                plt.figure(figsize=(10,6))
                shap.summary_plot(combined_shap, X_test, 
                                feature_names=self.feature_names,
                                plot_type="bar", show=False)
                plt.title(f"{model_name} 특성 중요도 (SHAP)")
                shap_summary_path = f"shap_summary_{model_name}.png"
                plt.savefig(shap_summary_path, bbox_inches='tight')
                plt.close()
                
                # Individual explanation (첫 번째 클래스 사용)
                plt.figure(figsize=(12,6))
                if isinstance(shap_vals, list):
                    individual_shap = shap_vals[1]  # 중위험군 클래스 사용
                else:
                    individual_shap = shap_vals
                
                # Force plot 대신 waterfall plot 사용 (더 안정적)
                shap.waterfall_plot(
                    shap.Explanation(
                        values=individual_shap[sample_index,:],
                        base_values=self.shap_explainers[model_name].expected_value[1] if isinstance(self.shap_explainers[model_name].expected_value, list) else self.shap_explainers[model_name].expected_value,
                        data=X_test.iloc[sample_index,:].values,
                        feature_names=self.feature_names
                    ),
                    show=False
                )
                plt.title(f"{model_name} 개별 설명 (샘플 {sample_index})")
                shap_force_path = f"shap_waterfall_{model_name}_{sample_index}.png"
                plt.savefig(shap_force_path, bbox_inches='tight')
                plt.close()
                
                shap_figures.extend([shap_summary_path, shap_force_path])
                print(f"✅ {model_name} SHAP 시각화 완료")
                
            except Exception as e:
                print(f"❌ {model_name} SHAP 시각화 실패: {e}")
                print(f"   상세 오류 정보:")
                import traceback
                traceback.print_exc()
        
        # LIME 시각화 (기존 코드 유지)
        lime_figures = []
        for model_name in self.lime_explainers:
            try:
                exp = self.lime_explainers[model_name].explain_instance(
                    X_test.iloc[sample_index].values,
                    self.models[model_name].predict_proba,
                    num_features=5
                )
                lime_path = f"lime_explanation_{model_name}_{sample_index}.png"
                fig = exp.as_pyplot_figure()
                plt.title(f"{model_name} LIME 설명 (샘플 {sample_index})")
                plt.savefig(lime_path, bbox_inches='tight')
                plt.close()
                lime_figures.append(lime_path)
                print(f"✅ {model_name} LIME 시각화 완료")
            except Exception as e:
                print(f"❌ {model_name} LIME 시각화 실패: {e}")
        
        return shap_figures, lime_figures

    def plot_results(self, X_test, y_test_dur, y_test_event, risk_test, risk_labels):
        """결과 시각화 (XAI 포함)"""
        fig = plt.figure(figsize=(25, 35))
        gs = fig.add_gridspec(5, 3)
        axes = [
            fig.add_subplot(gs[0, 0]),  # 생존 곡선
            fig.add_subplot(gs[0, 1]),  # 위험도 분포
            fig.add_subplot(gs[0, 2]),  # 생존 모델 성능
            fig.add_subplot(gs[1, 0]),  # 분류 Accuracy
            fig.add_subplot(gs[1, 1]),  # 분류 AUC
            fig.add_subplot(gs[1, 2]),  # RF 특성 중요도
            fig.add_subplot(gs[2, 0]),  # XGB 특성 중요도
            fig.add_subplot(gs[2, 1]),  # 생존율 히트맵
            fig.add_subplot(gs[2, 2]),  # 모델 성능 종합
            fig.add_subplot(gs[3, :]),  # SHAP 시각화
            fig.add_subplot(gs[4, :])   # LIME 시각화
        ]
        
        # 1. 위험도별 생존 곡선
        print("🔍 위험도별 생존 곡선 생성 중...")
        for risk_level in range(3):
            mask = (risk_test == risk_level)
            if mask.sum() > 5:
                kmf = KaplanMeierFitter()
                kmf.fit(y_test_dur[mask], y_test_event[mask], label=f'{risk_labels[risk_level]} (n={mask.sum()})')
                kmf.plot_survival_function(ax=axes[0])
        axes[0].set_title('위험도별 생존 곡선 (Kaplan-Meier)')
        axes[0].set_ylabel('생존 확률')
        axes[0].set_xlabel('시간 (일)')
        axes[0].legend()
        axes[0].grid(True, alpha=0.3)
        
        # 2. 위험도 분포
        risk_counts = pd.Series(risk_test).value_counts().sort_index()
        colors = ['lightgreen', 'gold', 'lightcoral']
        bars = axes[1].bar([risk_labels[i] for i in risk_counts.index], 
                          risk_counts.values, color=colors)
        axes[1].set_title('테스트 세트 위험도 분포')
        axes[1].set_ylabel('환자 수')
        for bar, value in zip(bars, risk_counts.values):
            axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
                        f'{value}명', ha='center', va='bottom', fontweight='bold')
        
        # 3. 생존 모델 성능 비교
        survival_models = ['Cox_Survival', 'RSF', 'GBSA']
        survival_c_indices = []
        survival_names = []
        for model_name in survival_models:
            if model_name in self.results and 'Test' in self.results[model_name]:
                c_index = self.results[model_name]['Test'].get('c_index', np.nan)
                if not np.isnan(c_index):
                    survival_c_indices.append(c_index)
                    survival_names.append(model_name)
        if survival_c_indices:
            bars = axes[2].bar(survival_names, survival_c_indices, 
                             color=['skyblue', 'lightcoral', 'lightgreen'][:len(survival_names)])
            axes[2].set_title('생존 모델 성능 (C-index)')
            axes[2].set_ylabel('C-index')
            axes[2].set_ylim(0.5, 1.0)
            axes[2].grid(True, alpha=0.3, axis='y')
            axes[2].tick_params(axis='x', rotation=45)
            for bar, value in zip(bars, survival_c_indices):
                axes[2].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                            f'{value:.3f}', ha='center', va='bottom', fontweight='bold')
        
        # 4. 분류 모델 성능 비교 (Accuracy)
        classification_models = ['Risk_RF', 'Logistic_Risk', 'XGBoost_Risk']
        classification_accuracies = []
        classification_names = []
        for model_name in classification_models:
            if model_name in self.results and 'Test' in self.results[model_name]:
                accuracy = self.results[model_name]['Test'].get('accuracy', np.nan)
                if not np.isnan(accuracy):
                    classification_accuracies.append(accuracy)
                    classification_names.append(model_name)
        if classification_accuracies:
            bars = axes[3].bar(classification_names, classification_accuracies, 
                             color=['lightgreen', 'gold', 'orange'][:len(classification_names)])
            axes[3].set_title('위험도 분류 모델 성능 (Accuracy)')
            axes[3].set_ylabel('Accuracy')
            axes[3].set_ylim(0, 1.0)
            axes[3].grid(True, alpha=0.3, axis='y')
            axes[3].tick_params(axis='x', rotation=45)
            for bar, value in zip(bars, classification_accuracies):
                axes[3].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                            f'{value:.3f}', ha='center', va='bottom', fontweight='bold')
        
        # 5. 분류 모델 성능 비교 (AUC)
        classification_aucs = []
        auc_names = []
        for model_name in classification_models:
            if model_name in self.results and 'Test' in self.results[model_name]:
                auc = self.results[model_name]['Test'].get('auc', np.nan)
                if not np.isnan(auc):
                    classification_aucs.append(auc)
                    auc_names.append(model_name)
        if classification_aucs:
            bars = axes[4].bar(auc_names, classification_aucs, 
                             color=['lightgreen', 'gold', 'orange'][:len(auc_names)])
            axes[4].set_title('위험도 분류 모델 성능 (AUC)')
            axes[4].set_ylabel('AUC')
            axes[4].set_ylim(0, 1.0)
            axes[4].grid(True, alpha=0.3, axis='y')
            axes[4].tick_params(axis='x', rotation=45)
            for bar, value in zip(bars, classification_aucs):
                axes[4].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                            f'{value:.3f}', ha='center', va='bottom', fontweight='bold')
        
        # 6. RF 특성 중요도
        if 'Risk_RF' in self.models:
            try:
                importance = self.models['Risk_RF'].feature_importances_
                feature_importance_df = pd.DataFrame({
                    'feature': self.feature_names,
                    'importance': importance
                }).sort_values('importance', ascending=True).tail(10)
                bars = axes[5].barh(range(len(feature_importance_df)), 
                                   feature_importance_df['importance'],
                                   color='lightblue')
                axes[5].set_yticks(range(len(feature_importance_df)))
                axes[5].set_yticklabels(feature_importance_df['feature'], fontsize=10)
                axes[5].set_title('특성 중요도 (Random Forest)', fontsize=12, fontweight='bold')
                axes[5].set_xlabel('중요도')
                axes[5].grid(True, alpha=0.3, axis='x')
            except Exception as e:
                print(f"❌ Random Forest 특성 중요도 시각화 실패: {e}")
                axes[5].text(0.5, 0.5, 'RF 특성 중요도\n계산 실패', 
                            ha='center', va='center', transform=axes[5].transAxes,
                            fontsize=12, fontweight='bold')
        
        # 7. XGB 특성 중요도
        if 'XGBoost_Risk' in self.models:
            try:
                importance = self.models['XGBoost_Risk'].feature_importances_
                feature_importance_df = pd.DataFrame({
                    'feature': self.feature_names,
                    'importance': importance
                }).sort_values('importance', ascending=True).tail(10)
                bars = axes[6].barh(range(len(feature_importance_df)), 
                                   feature_importance_df['importance'],
                                   color='orange')
                axes[6].set_yticks(range(len(feature_importance_df)))
                axes[6].set_yticklabels(feature_importance_df['feature'], fontsize=10)
                axes[6].set_title('특성 중요도 (XGBoost)', fontsize=12, fontweight='bold')
                axes[6].set_xlabel('중요도')
                axes[6].grid(True, alpha=0.3, axis='x')
            except Exception as e:
                print(f"❌ XGBoost 특성 중요도 시각화 실패: {e}")
                axes[6].text(0.5, 0.5, 'XGBoost 특성 중요도\n계산 실패', 
                            ha='center', va='center', transform=axes[6].transAxes,
                            fontsize=12, fontweight='bold')
        
        # 8. 위험도별 생존율 요약
        survival_summary = []
        time_points = [365, 1095, 1825]  # 1년, 3년, 5년
        for risk_level in range(3):
            mask = (risk_test == risk_level)
            if mask.sum() > 5:
                kmf = KaplanMeierFitter()
                kmf.fit(y_test_dur[mask], y_test_event[mask])
                survival_rates = []
                for time_point in time_points:
                    try:
                        survival_rate = kmf.survival_function_at_times(time_point).values[0]
                        survival_rates.append(survival_rate * 100)
                    except:
                        survival_rates.append(np.nan)
                survival_summary.append(survival_rates)
            else:
                survival_summary.append([np.nan, np.nan, np.nan])
        survival_df = pd.DataFrame(survival_summary, 
                                  index=[risk_labels[i] for i in range(3)],
                                  columns=['1년', '3년', '5년'])
        sns.heatmap(survival_df, annot=True, fmt='.1f', cmap='RdYlGn', 
                   ax=axes[7], cbar_kws={'label': '생존율 (%)'})
        axes[7].set_title('위험도별 생존율 요약')
        axes[7].set_ylabel('위험도 그룹')
        
        # 9. 모델 성능 종합 비교
        all_models_performance = {}
        for model_name in ['Cox_Survival', 'RSF', 'GBSA']:
            if model_name in self.results and 'Test' in self.results[model_name]:
                c_index = self.results[model_name]['Test'].get('c_index', np.nan)
                if not np.isnan(c_index):
                    all_models_performance[f'{model_name}_C-index'] = c_index
        for model_name in ['Risk_RF', 'Logistic_Risk', 'XGBoost_Risk']:
            if model_name in self.results and 'Test' in self.results[model_name]:
                accuracy = self.results[model_name]['Test'].get('accuracy', np.nan)
                if not np.isnan(accuracy):
                    all_models_performance[f'{model_name}_Accuracy'] = accuracy
        if all_models_performance:
            model_names = list(all_models_performance.keys())
            model_scores = list(all_models_performance.values())
            bars = axes[8].bar(range(len(model_names)), model_scores, 
                             color=['skyblue', 'lightcoral', 'lightgreen', 
                                   'gold', 'orange', 'pink'][:len(model_names)])
            axes[8].set_title('전체 모델 성능 종합')
            axes[8].set_ylabel('성능 점수')
            axes[8].set_xticks(range(len(model_names)))
            axes[8].set_xticklabels(model_names, rotation=45, ha='right')
            axes[8].grid(True, alpha=0.3, axis='y')
            for bar, value in zip(bars, model_scores):
                axes[8].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                            f'{value:.3f}', ha='center', va='bottom', fontweight='bold', fontsize=8)
        
        # 10. SHAP 시각화
        try:
            model_name = 'XGBoost_Risk' if 'XGBoost_Risk' in self.shap_explainers else 'Risk_RF'
            if model_name:
                img = plt.imread(f"shap_summary_{model_name}.png")
                axes[9].imshow(img)
                axes[9].axis('off')
                axes[9].set_title('SHAP 전역 설명', fontsize=12)
        except Exception as e:
            print(f"❌ SHAP 시각화 로드 실패: {e}")
            axes[9].text(0.5, 0.5, 'SHAP 시각화 불러오기 실패', 
                        ha='center', va='center', transform=axes[9].transAxes,
                        fontsize=12, fontweight='bold')
        
        # 11. LIME 시각화
        try:
            model_name = 'XGBoost_Risk' if 'XGBoost_Risk' in self.lime_explainers else 'Risk_RF'
            if model_name:
                img = plt.imread(f"lime_explanation_{model_name}_0.png")
                axes[10].imshow(img)
                axes[10].axis('off')
                axes[10].set_title('LIME 개별 설명 (첫 번째 환자)', fontsize=12)
        except Exception as e:
            print(f"❌ LIME 시각화 로드 실패: {e}")
            axes[10].text(0.5, 0.5, 'LIME 시각화 불러오기 실패', 
                         ha='center', va='center', transform=axes[10].transAxes,
                         fontsize=12, fontweight='bold')
        
        plt.tight_layout()
        save_path = "tcga_lihc_risk_stratification_with_xai.png"
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"📁 시각화 결과 저장: {save_path}")
        
        return True
    
    def generate_report(self, risk_labels):
        """결과 보고서 생성"""
        print("\n📋 9. 결과 보고서 생성")
        
        report = []
        report.append("="*60)
        report.append("간암 환자 위험도 계층화 모델 분석 결과")
        report.append("="*60)
        report.append(f"분석 일시: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        report.append(f"데이터 경로: {self.data_path}")
        report.append("")
        
        # 데이터 요약
        report.append("📊 데이터 요약")
        report.append("-" * 30)
        report.append(f"총 환자 수: {len(self.processed_df)}")
        report.append(f"사망 환자 수: {self.processed_df['event'].sum()}")
        report.append(f"사망률: {self.processed_df['event'].mean()*100:.1f}%")
        report.append(f"중간 생존 시간: {self.processed_df['duration'].median():.0f}일")
        report.append(f"사용된 특성 수: {len(self.feature_names)}")
        report.append("")
        
        # 위험도 분포
        report.append("🎯 위험도 계층화 결과")
        report.append("-" * 30)
        for i, label in enumerate(risk_labels):
            count = len(self.processed_df)  # 임시값
            report.append(f"{label}: 분석 완료")
        report.append("")
        
        # 모델 성능
        report.append("🤖 모델 성능")
        report.append("-" * 30)
        
        # 생존 모델 성능
        report.append("\n생존 예측 모델 (C-index):")
        survival_models = ['Cox_Survival', 'RSF', 'GBSA']
        for model_name in survival_models:
            if model_name in self.results:
                for dataset in ['Train', 'Validation', 'Test']:
                    if dataset in self.results[model_name]:
                        metrics = self.results[model_name][dataset]
                        if 'c_index' in metrics:
                            report.append(f"  {model_name} {dataset}: C-index = {metrics['c_index']:.3f}")
        
        # 분류 모델 성능
        report.append("\n위험度 분류 모델 (Accuracy):")
        classification_models = ['Risk_RF', 'Logistic_Risk', 'XGBoost_Risk']
        for model_name in classification_models:
            if model_name in self.results:
                for dataset in ['Train', 'Validation', 'Test']:
                    if dataset in self.results[model_name]:
                        metrics = self.results[model_name][dataset]
                        if 'accuracy' in metrics:
                            report.append(f"  {model_name} {dataset}: Accuracy = {metrics['accuracy']:.3f}")
        
        # 최고 성능 모델
        test_performances = {}
        for model_name in self.results:
            if 'Test' in self.results[model_name]:
                metrics = self.results[model_name]['Test']
                if 'c_index' in metrics:
                    test_performances[f"{model_name}_C-index"] = metrics['c_index']
                elif 'accuracy' in metrics:
                    test_performances[f"{model_name}_Accuracy"] = metrics['accuracy']
        
        if test_performances:
            best_model = max(test_performances, key=test_performances.get)
            report.append(f"\n🏆 최고 성능 모델: {best_model} (점수: {test_performances[best_model]:.3f})")
        
        report.append("")
        report.append("ℹ️  사용된 모델: Cox, RSF, GBSA, Random Forest, Logistic Regression, XGBoost")
        report.append("ℹ️  DeepSurv는 안정성을 위해 제외됨")
        report.append("ℹ️  XAI: SHAP, LIME 적용")
        report.append("")
        report.append("="*60)
        
        # 보고서 저장
        report_text = "\n".join(report)
        with open("tcga_lihc_risk_stratification_report.txt", "w", encoding="utf-8") as f:
            f.write(report_text)
        
        print("📁 보고서 저장: tcga_lihc_risk_stratification_report.txt")
        print("\n" + report_text)
        
        return report_text
    
    def save_models(self):
        """모델 저장"""
        print("\n💾 10. 모델 저장")
        
        for model_name, model in self.models.items():
            try:
                filename = f"tcga_lihc_risk_{model_name.lower()}_model.pkl"
                joblib.dump(model, filename)
                print(f"✅ {model_name} 모델 저장: {filename}")
            except Exception as e:
                print(f"❌ {model_name} 모델 저장 실패: {e}")
        
        return True
    
    def run_complete_analysis(self):
        """전체 분석 실행 (XAI 포함)"""
        print("🎯 간암 위험도 계층화 모델 전체 분석 시작 (XAI 포함)")
        
        try:
            # 1. 데이터 로드
            if not self.load_and_explore_data():
                return False
            
            # 2. 데이터 전처리
            if not self.preprocess_data():
                return False
            
            # 3. 특성 준비
            X, y_structured, y_duration, y_event, scaler, label_encoders = self.prepare_features()
            
            # 4. 위험도 점수 계산
            total_scores, risk_categories, risk_labels = self.create_risk_scores(X)
            
            # 5. 데이터 분할
            (X_train, X_val, X_test, 
             y_train_struct, y_val_struct, y_test_struct,
             y_train_dur, y_val_dur, y_test_dur,
             y_train_event, y_val_event, y_test_event,
             risk_train, risk_val, risk_test) = self.split_data(X, y_structured, y_duration, y_event, risk_categories)
            
            # 6. 모델 훈련
            if not self.train_models(X_train, X_val, X_test,
                                   y_train_struct, y_val_struct, y_test_struct,
                                   y_train_dur, y_val_dur, y_test_dur,
                                   y_train_event, y_val_event, y_test_event,
                                   risk_train, risk_val, risk_test):
                return False
            
            # 7. 모델 평가
            if not self.evaluate_models(X_train, X_val, X_test,
                                       y_train_struct, y_val_struct, y_test_struct,
                                       y_train_dur, y_val_dur, y_test_dur,
                                       y_train_event, y_val_event, y_test_event,
                                       risk_train, risk_val, risk_test):
                return False
            
            # 8. XAI 설명 생성
            self.explain_models(X_train, X_test)
            
            # 9. XAI 시각화 생성
            self.generate_xai_visualizations(X_test)
            
            # 10. 시각화 (XAI 포함)
            if not self.plot_results(X_test, y_test_dur, y_test_event, risk_test, risk_labels):
                return False
            
            # 11. 보고서 생성
            self.generate_report(risk_labels)
            
            # 12. 모델 저장
            self.save_models()
            
            print("\n🎉 전체 분석 완료!")
            print(f"⏰ 완료 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
            
            return True
            
        except Exception as e:
            print(f"\n❌ 분석 중 오류 발생: {e}")
            import traceback
            traceback.print_exc()
            return False

# 실행
if __name__ == "__main__":
    data_path = r"C:\Users\02\Documents\GDCdata_liver\clinical_data_liver.csv"
    risk_model = LiverCancerRiskStratification(data_path)
    success = risk_model.run_complete_analysis()
    
    if success:
        print("\n✨ 모든 분석이 성공적으로 완료되었습니다!")
        print("📁 생성된 파일들:")
        print("   - tcga_lihc_risk_stratification_with_xai.png (시각화 결과)")
        print("   - tcga_lihc_risk_stratification_report.txt (분석 보고서)")
        print("   - tcga_lihc_risk_*_model.pkl (훈련된 모델들)")
        print("   - shap_*.png (SHAP 설명)")
        print("   - lime_*.png (LIME 설명)")
        print("\n🔬 포함된 모델들:")
        print("   생존 예측: Cox, RSF, GBSA")
        print("   위험도 분류: Random Forest, Logistic Regression, XGBoost")
        print("   XAI: SHAP, LIME 적용 (DeepSurv는 안정성을 위해 제외됨)")
    else:
        print("\n❌ 분석 중 문제가 발생했습니다. 로그를 확인해주세요.")


✅ 한글 폰트 설정 완료
🚀 간암 위험도 계층화 모델 초기화 (XAI 포함)
📁 데이터 경로: C:\Users\02\Documents\GDCdata_liver\clinical_data_liver.csv
⏰ 시작 시간: 2025-06-13 14:50:00
🎯 간암 위험도 계층화 모델 전체 분석 시작 (XAI 포함)

📊 1. 데이터 로드 및 탐색
✅ 데이터 로드 성공: 377행 × 87열
📈 데이터 기본 정보:
   - 총 환자 수: 377
   - 총 컬럼 수: 87
   - 생존 환자: 245명
   - 사망 환자: 132명
   - 사망률: 35.0%

🔧 2. 데이터 전처리
✅ 사용 가능한 컬럼: 22개
🔄 생존 변수 생성 중...
✅ 유효한 생존 데이터: 372명
   - 사망 이벤트: 132건
   - 중간 생존 시간: 602일

📋 결측값 분석:
   - days_to_death: 240개 (64.5%)
   - child_pugh_classification: 47개 (12.6%)
   - ishak_fibrosis_score: 96개 (25.8%)
   - ajcc_pathologic_stage: 24개 (6.5%)
   - ajcc_pathologic_t: 2개 (0.5%)
   - ajcc_pathologic_n: 1개 (0.3%)
   - tumor_grade: 5개 (1.3%)
   - age_at_diagnosis: 3개 (0.8%)
   - residual_disease: 7개 (1.9%)
   - year_of_diagnosis: 2개 (0.5%)

🎯 3. 특성 준비 및 인코딩
📊 초기 특성 개수: 19
📊 샘플 개수: 372
🔤 범주형 변수: 17개
🔢 수치형 변수: 2개
🔄 결측값 처리 중...

   🔍 child_pugh_classification 처리:
      - 전처리 전 분포: {'A': 222, 'Unknown': 81, nan: 47}
      - 'Unknown' 값 유지 (임상적 의미 있음)
      - 결

<Figure size 1200x600 with 0 Axes>

<Figure size 1200x600 with 0 Axes>

## SHAP 특성 중요도 분석

1) 가장 중요한 예측 인자들 (XGBoost 기준):
1. **Gender (성별)**: 0.8 - 압도적으로 가장 중요한 인자
2. **AJCC Pathologic Stage (병기)**: 0.6 - 두 번째로 중요
3. **AJCC Pathologic T (T 병기)**: 0.4 - 종양 크기/침범도
4. **Child-Pugh Classification**: 0.35 - 간기능 상태
5. **Age at Diagnosis (진단 시 연령)**: 0.25 - 연령 요인

## LIME 개별 환자 설명 (샘플 0)

1) 해당 환자의 위험도에 영향을 미친 요인들:
- **AJCC Pathologic Stage ≤ -0.70**: 저위험 방향으로 강하게 기여 (녹색)
- **Child-Pugh Classification ≤ -0.58**: 저위험 방향으로 기여 (녹색)
- **Gender**: 고위험 방향으로 기여 (빨간색, -1.44 ≤ gender ≤ 0.69)
- **Residual Disease**: 고위험 방향으로 기여 (빨간색)
- **Tumor Grade**: 고위험 방향으로 기여 (빨간색)

## 임상적 해석

1) 성별의 압도적 영향력:
- 검색 결과[1]의 한국인 간암 연구와 일치하는 결과
- 남성이 여성보다 간암 위험도가 현저히 높음
- 이는 생활습관, 호르몬, B형/C형 간염 유병률 차이 등이 복합적으로 작용

2) 병기 시스템의 중요성:
- AJCC 병기 시스템이 예상대로 핵심 예측 인자로 확인
- T 병기(종양 크기/침범도)가 특히 중요
- 조기 발견의 중요성을 재확인

3) 간기능 평가의 역할:
- Child-Pugh 분류가 4번째로 중요한 인자
- 간암 환자에서 기저 간기능이 예후에 미치는 영향 확인

## 모델 성능 분석

1) 성능:
- **위험도 분류 정확도**: 93.3% (테스트 세트)
- **AUC**: 0.978 (XGBoost) - 매우 우수한 성능
- **생존 예측 C-index**: 0.645 (RSF) - 양호한 성능

## 위험도 계층화 결과

2) 분포:
- **저위험군**: 37명 (9.9%)
- **중위험군**: 288명 (77.4%) 
- **고위험군**: 47명 (12.6%)

## XAI의 임상적 가치

1) 개별화된 치료 계획:
- LIME 결과를 통해 각 환자별로 어떤 인자가 위험도에 가장 큰 영향을 미쳤는지 명확히 파악
- 의료진이 해당 인자들을 집중적으로 관리할 수 있는 근거 제공

2) 성별 기반 맞춤 관리:
- 남성 환자에서 더 적극적인 선별검사와 추적관찰 필요
- 여성 환자에서는 다른 위험 인자들에 더 주목

3) 병기별 차별화 전략:
- 조기 병기 환자: 정기적 추적관찰
- 진행성 병기 환자: 집중적 치료 및 모니터링
